In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Paths
data_path = Path.cwd().parent / "data"
gsddat_folder = data_path / "gsds" / "val"
df = pd.read_csv(gsddat_folder / "gsd-metrics.csv")

In [ ]:
df

In [ ]:
fig, axs = plt.subplots(nrows=3, sharex=True)

ious = [100 * df[df.GSD == gsd].iou for gsd in df.GSD.unique()]
accs = [100 * df[df.GSD == gsd].acc for gsd in df.GSD.unique()]
f1s = [100 * df[df.GSD == gsd].f1 for gsd in df.GSD.unique()]

m_IOU = [iou.mean() for iou in ious]
m_acc = [acc.mean() for acc in accs]
m_f1 = [f1.mean() for f1 in f1s]

vio_ious = axs[0].violinplot(ious, showmeans=False)
vio_accs = axs[1].violinplot(accs, showmeans=False)
vio_f1s = axs[2].violinplot(f1s, showmeans=False)
xlabs = ["$GSD$ =\n0.8-1.3 mm", "$1.5\\times GSD$", "$2.25\\times GSD$", "$3.375\\times GSD$"]

for pc_iou in vio_ious["bodies"]:
    pc_iou.set_facecolor("#4248ad")
    pc_iou.set_edgecolor("#4248ad")
    pc_iou.set_alpha(1)

for pc_acc in vio_accs["bodies"]:
    pc_acc.set_facecolor("#cb416b")
    pc_acc.set_edgecolor("#cb416b")
    pc_acc.set_alpha(1)

for pc_f1 in vio_f1s["bodies"]:
    pc_f1.set_facecolor("#ff9408")
    pc_f1.set_edgecolor("#ff9408")
    pc_f1.set_alpha(1)

axs[0].plot(np.arange(1, 5), m_IOU, "k")
axs[1].plot(np.arange(1, 5), m_acc, "k")
axs[2].plot(np.arange(1, 5), m_f1, "k")

axs[2].set_xticks([1, 2, 3, 4], xlabs)
axs[2].set_xlabel("Ground Sample Distance (m)")

axs[0].set_ylabel("IoU (%)")
axs[1].set_ylabel("Pixel Accuracy (%)")
axs[2].set_ylabel("F1 score (%)")

axs[0].set_ylim(bottom=0)
axs[1].set_ylim(bottom=0)
axs[2].set_ylim(bottom=0)



In [ ]:
fig, axs = plt.subplots(nrows=3, sharex=True)

ious = [100 * df[df.GSD == gsd].iou for gsd in df.GSD.unique()]
accs = [100 * df[df.GSD == gsd].acc for gsd in df.GSD.unique()]
f1s = [100 * df[df.GSD == gsd].f1 for gsd in df.GSD.unique()]

m_IOU = [iou.mean() for iou in ious]
m_acc = [acc.mean() for acc in accs]
m_f1 = [f1.mean() for f1 in f1s]

xlabs = [
    "$GSD$ =\n0.8-1.3 mm",
    "$1.5\\times GSD$",
    "$2.25\\times GSD$",
    "$3.375\\times GSD$",
]

axs[0].plot(np.arange(1, 5), m_IOU, marker="o", linestyle="", color="xkcd:grass green")
axs[1].plot(np.arange(1, 5), m_acc, marker="o", linestyle="", color="xkcd:darkish blue")
axs[2].plot(np.arange(1, 5), m_f1, marker="o", linestyle="", color="xkcd:orange")

axs[2].set_xticks([1, 2, 3, 4], xlabs)
axs[2].set_xlabel("Ground Sample Distance (mm)")

axs[0].set_ylabel("mIoU (%)", rotation=0, ha="right", va="center")
axs[1].set_ylabel("Average pixel\naccuracy (%)", rotation=0, ha="right", va="center")
axs[2].set_ylabel("F1 score (%)", rotation=0, ha="right", va="center")

axs[0].yaxis.set_label_coords(-0.1, 0.5)
axs[1].yaxis.set_label_coords(-0.1, 0.5)
axs[2].yaxis.set_label_coords(-0.1, 0.5)

axs[0].set_ylim(bottom=0, top=30)
axs[1].set_ylim(bottom=0, top=50)
axs[2].set_ylim(bottom=0, top=40)
